<a href="https://colab.research.google.com/github/jyotidabass/NLP-Projects/blob/main/finance_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install required libraries
!pip install gradio
!pip install groq

import gradio as gr
import groq
import os
import json
from typing import Dict, List
import pandas as pd
from datetime import datetime
import getpass

# Ask for GROQ API key as a secret
GROQ_API_KEY = getpass.getpass("Enter your GROQ API key:")


class FinanceAIAgent:
    def __init__(self, api_key: str):
        self.client = groq.Client(api_key=api_key)
        self.model = "llama-3.3-70b-versatile"
        self.conversation_history = []

    def generate_response(self, prompt: str, context: str = "") -> str:
        # Combine context and prompt
        full_prompt = f"{context}\n\nUser: {prompt}\nAssistant:"

        try:
            chat_completion = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": full_prompt}],
                temperature=0.7,
                max_tokens=1000
            )
            return chat_completion.choices[0].message.content
        except Exception as e:
            return f"Error generating response: {str(e)}"

    def analyze_portfolio(self, portfolio_data: str) -> str:
        prompt = f"""Analyze the following investment portfolio and provide insights:
        {portfolio_data}
        Include:
        1. Risk assessment
        2. Diversification analysis
        3. Recommendations for rebalancing
        4. Potential areas of concern"""
        return self.generate_response(prompt)

    def financial_planning(self, income: float, expenses: List[Dict], goals: List[str]) -> str:
        prompt = f"""Create a financial plan based on:
        Income: ${income}
        Monthly Expenses: {json.dumps(expenses, indent=2)}
        Financial Goals: {json.dumps(goals, indent=2)}

        Provide:
        1. Budget breakdown
        2. Savings recommendations
        3. Investment strategies
        4. Timeline for achieving goals"""
        return self.generate_response(prompt)

    def market_analysis(self, ticker: str, timeframe: str) -> str:
        prompt = f"""Provide a detailed market analysis for {ticker} over {timeframe} timeframe.
        Include:
        1. Technical analysis perspectives
        2. Fundamental factors
        3. Market sentiment
        4. Risk factors
        5. Potential catalysts"""
        return self.generate_response(prompt)

def create_finance_ai_interface():
    agent = FinanceAIAgent(api_key=GROQ_API_KEY)

    with gr.Blocks(title="Finance AI Assistant") as interface:
        gr.Markdown("# Finance AI Assistant")

        with gr.Tab("Portfolio Analysis"):
            portfolio_input = gr.Textbox(
                label="Enter portfolio details (ticker symbols and allocations)",
                placeholder="AAPL: 25%, MSFT: 25%, GOOGL: 25%, AMZN: 25%"
            )
            portfolio_button = gr.Button("Analyze Portfolio")
            portfolio_output = gr.Textbox(label="Analysis Results")

            portfolio_button.click(
                fn=agent.analyze_portfolio,
                inputs=[portfolio_input],
                outputs=portfolio_output
            )

        with gr.Tab("Financial Planning"):
            with gr.Row():
                income_input = gr.Number(label="Monthly Income ($)")

            with gr.Row():
                expenses_input = gr.Dataframe(
                    headers=["Category", "Amount"],
                    datatype=["str", "number"],
                    label="Monthly Expenses"
                )

            goals_input = gr.Textbox(
                label="Financial Goals (one per line)",
                placeholder="1. Save for retirement\n2. Buy a house\n3. Start a business"
            )

            planning_button = gr.Button("Generate Financial Plan")
            planning_output = gr.Textbox(label="Financial Plan")

            def process_financial_plan(income, expenses_df, goals):
                expenses = expenses_df.to_dict('records')
                goals_list = [g.strip() for g in goals.split('\n') if g.strip()]
                return agent.financial_planning(income, expenses, goals_list)

            planning_button.click(
                fn=process_financial_plan,
                inputs=[income_input, expenses_input, goals_input],
                outputs=planning_output
            )

        with gr.Tab("Market Analysis"):
            with gr.Row():
                ticker_input = gr.Textbox(label="Stock Ticker")
                timeframe_input = gr.Dropdown(
                    choices=["1 day", "1 week", "1 month", "3 months", "1 year"],
                    label="Timeframe"
                )

            market_button = gr.Button("Analyze Market")
            market_output = gr.Textbox(label="Market Analysis")

            market_button.click(
                fn=agent.market_analysis,
                inputs=[ticker_input, timeframe_input],
                outputs=market_output
            )

        with gr.Tab("AI Chat"):
            chatbot = gr.Chatbot()
            msg = gr.Textbox(label="Ask anything about finance")
            clear = gr.Button("Clear")

            def respond(message, history):
                history.append((message, agent.generate_response(message)))
                return "", history

            msg.submit(respond, [msg, chatbot], [msg, chatbot])
            clear.click(lambda: None, None, chatbot, queue=False)

    return interface

# Launch the interface
interface = create_finance_ai_interface()
interface.launch()

Enter your GROQ API key:··········


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fc857e0a1b1dab1102.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
